In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import holidays
import itertools
from sklearn.linear_model import LinearRegression, Ridge
# from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, max_error, mean_absolute_error, r2_score
import time

# 1. Import Data

In [3]:
df_data_w_clusters = pd.read_csv('/content/drive/My Drive/MIE498 Thesis/Share-Bike-Station-Clustering-and-Usage-Prediction/station_data_w_clusters.csv', index_col=0)

In [4]:
df_data_w_clusters.head()

,station_id,station_na,lat,lon,neighborhood_id,FSA_code,da_id,cluster
0,7021,Bay St / Albert St,43.653264,-79.382458,76,M5G,35204616.0,0
1,7160,King St W / Tecumseth St,43.643333,-79.405556,82,M5V,35204142.0,35
2,7012,Elizabeth St / Edward St (Bus Terminal),43.656026,-79.385327,76,M5G,35204607.0,16
3,7041,Edward St / Yonge St,43.656729,-79.382736,76,M5G,35204608.0,16
4,7275,Queen St W / James St,43.652276,-79.380701,76,M5G,35204616.0,34


In [5]:
# Import ridership data
data_dir = '/content/drive/My Drive/MIE498 Thesis/0_Data'
merged_bike_data_2019 = pd.read_csv("{}/ridership_2019_with_bike_stations_info_20200930.csv".format(data_dir), header=0)
print(merged_bike_data_2019.shape)
merged_bike_data_2019['End Day of Year'] = merged_bike_data_2019.apply(lambda row: datetime.strptime(row['End Time'], "%Y-%m-%d %H:%M:%S").timetuple().tm_yday, axis=1)
print(merged_bike_data_2019.shape)
merged_bike_data_2019.head()

(2438720, 26)
(2438720, 27)


,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type,Bike Id,...,End Hour,End Day of Week,End Holiday,Start Day of Year,Start Week of Year,Start Lat,Start Lon,End Lat,End Lon,End Day of Year
0,4581278.0,1547.0,7021.0,2019-01-01 00:08:00,Bay St / Albert St,7233.0,2019-01-01 00:33:00,King / Cowan Ave - SMART,Annual Member,1296.0,...,0.0,1.0,1.0,1.0,1.0,43.653264,-79.382458,43.637922,-79.431734,1
1,4586979.0,1243.0,7021.0,2019-01-03 17:13:00,Bay St / Albert St,7233.0,2019-01-03 17:34:00,King / Cowan Ave - SMART,Annual Member,3168.0,...,17.0,3.0,0.0,3.0,1.0,43.653264,-79.382458,43.637922,-79.431734,3
2,4591641.0,1112.0,7021.0,2019-01-04 21:42:00,Bay St / Albert St,7233.0,2019-01-04 22:00:00,King / Cowan Ave - SMART,Annual Member,3512.0,...,22.0,4.0,0.0,4.0,1.0,43.653264,-79.382458,43.637922,-79.431734,4
3,4594328.0,1156.0,7021.0,2019-01-05 23:21:00,Bay St / Albert St,7233.0,2019-01-05 23:40:00,King / Cowan Ave - SMART,Annual Member,212.0,...,23.0,5.0,0.0,5.0,1.0,43.653264,-79.382458,43.637922,-79.431734,5
4,4596322.0,1068.0,7021.0,2019-01-06 21:42:00,Bay St / Albert St,7233.0,2019-01-06 22:00:00,King / Cowan Ave - SMART,Annual Member,3279.0,...,22.0,6.0,0.0,6.0,1.0,43.653264,-79.382458,43.637922,-79.431734,6


In [6]:
merged_data = merged_bike_data_2019.copy()
merged_data = merged_data.merge(df_data_w_clusters[['station_id', 'cluster']], how='left', left_on='Start Station Id', right_on='station_id').drop('station_id', axis=1)
merged_data.rename({'cluster': 'Start Cluster'}, axis=1, inplace=True)
merged_data = merged_data.merge(df_data_w_clusters[['station_id', 'cluster']], how='left', left_on='End Station Id', right_on='station_id').drop('station_id', axis=1)
merged_data.rename({'cluster': 'End Cluster'}, axis=1, inplace=True)

In [7]:
merged_data.isnull().sum()

Trip Id               0
Trip Duration         0
Start Station Id      0
Start Time            0
Start Station Name    0
End Station Id        0
End Time              0
End Station Name      0
User Type             0
Bike Id               0
Start Year            0
Start Month           0
Start Hour            0
Start Day of Week     0
Start Holiday         0
End Year              0
End Month             0
End Hour              0
End Day of Week       0
End Holiday           0
Start Day of Year     0
Start Week of Year    0
Start Lat             0
Start Lon             0
End Lat               0
End Lon               0
End Day of Year       0
Start Cluster         0
End Cluster           0
dtype: int64

In [8]:
merged_data.columns

Index(['Trip Id', 'Trip Duration', 'Start Station Id', 'Start Time',
       'Start Station Name', 'End Station Id', 'End Time', 'End Station Name',
       'User Type', 'Bike Id', 'Start Year', 'Start Month', 'Start Hour',
       'Start Day of Week', 'Start Holiday', 'End Year', 'End Month',
       'End Hour', 'End Day of Week', 'End Holiday', 'Start Day of Year',
       'Start Week of Year', 'Start Lat', 'Start Lon', 'End Lat', 'End Lon',
       'End Day of Year', 'Start Cluster', 'End Cluster'],
      dtype='object')

In [9]:
merged_data['Start Time'] = merged_data.apply(lambda row: datetime.strptime(row['Start Time'], "%Y-%m-%d %H:%M:%S"), axis=1)
merged_data['Start Day'] = merged_data.apply(lambda row: row['Start Time'].day, axis=1)

In [10]:
merged_data['End Time'] = merged_data.apply(lambda row: datetime.strptime(row['End Time'], "%Y-%m-%d %H:%M:%S"), axis=1)
merged_data['End Day'] = merged_data.apply(lambda row: row['End Time'].day, axis=1)

In [11]:
merged_data.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type,Bike Id,...,Start Week of Year,Start Lat,Start Lon,End Lat,End Lon,End Day of Year,Start Cluster,End Cluster,Start Day,End Day
0,4581278.0,1547.0,7021.0,2019-01-01 00:08:00,Bay St / Albert St,7233.0,2019-01-01 00:33:00,King / Cowan Ave - SMART,Annual Member,1296.0,...,1.0,43.653264,-79.382458,43.637922,-79.431734,1,0,32,1,1
1,4586979.0,1243.0,7021.0,2019-01-03 17:13:00,Bay St / Albert St,7233.0,2019-01-03 17:34:00,King / Cowan Ave - SMART,Annual Member,3168.0,...,1.0,43.653264,-79.382458,43.637922,-79.431734,3,0,32,3,3
2,4591641.0,1112.0,7021.0,2019-01-04 21:42:00,Bay St / Albert St,7233.0,2019-01-04 22:00:00,King / Cowan Ave - SMART,Annual Member,3512.0,...,1.0,43.653264,-79.382458,43.637922,-79.431734,4,0,32,4,4
3,4594328.0,1156.0,7021.0,2019-01-05 23:21:00,Bay St / Albert St,7233.0,2019-01-05 23:40:00,King / Cowan Ave - SMART,Annual Member,212.0,...,1.0,43.653264,-79.382458,43.637922,-79.431734,5,0,32,5,5
4,4596322.0,1068.0,7021.0,2019-01-06 21:42:00,Bay St / Albert St,7233.0,2019-01-06 22:00:00,King / Cowan Ave - SMART,Annual Member,3279.0,...,1.0,43.653264,-79.382458,43.637922,-79.431734,6,0,32,6,6


In [12]:
merged_data.shape

(2438720, 31)

In [13]:
# Import weather data
df_weather = pd.read_csv('/content/drive/My Drive/MIE498 Thesis/Share-Bike-Station-Clustering-and-Usage-Prediction/toronto_weather_2019.csv', index_col=None)
df_weather['Month'] = df_weather['Month'].astype("float64")
df_weather['Day'] = df_weather['Day'].astype("float64")
df_weather['Hour'] = df_weather['Hour'].astype("float64")
df_weather['Day of Week'] = df_weather['Day of Week'].astype("float64")
df_weather.head()

,Month,Day,Hour,Day of Week,Temperature (Celsius),Precipitation Amount (mm)
0,1.0,1.0,0.0,1.0,5.1,0.5
1,1.0,1.0,1.0,1.0,6.2,0.5
2,1.0,1.0,2.0,1.0,5.6,0.0
3,1.0,1.0,3.0,1.0,3.2,0.4
4,1.0,1.0,4.0,1.0,2.7,0.0


In [14]:
merged_data = merged_data.merge(df_weather[['Month', 'Day', 'Hour', 'Temperature (Celsius)']], how='left', left_on=['Start Month', 'Start Day', 'Start Hour'], right_on=['Month', 'Day', 'Hour']).drop(['Month', 'Day', 'Hour'], axis=1)
merged_data.rename({'Temperature (Celsius)': 'Start Temp'}, axis=1, inplace=True)

In [15]:
merged_data = merged_data.merge(df_weather[['Month', 'Day', 'Hour', 'Temperature (Celsius)']], how='left', left_on=['End Month', 'End Day', 'End Hour'], right_on=['Month', 'Day', 'Hour']).drop(['Month', 'Day', 'Hour'], axis=1)
merged_data.rename({'Temperature (Celsius)': 'End Temp'}, axis=1, inplace=True)

In [16]:
# check-out and check-in data split
checkout_data = merged_data[['Trip Id', 'Start Station Id', 'Start Time', 'Start Station Name', 'Start Year', 'Start Month', 'Start Hour',
       'Start Day of Week', 'Start Holiday', 'Start Day of Year', 'Start Week of Year', 'Start Lat', 'Start Lon', 'Start Cluster',
       'Start Temp']]
checkin_data = merged_data[['Trip Id', 'End Station Id', 'End Time', 'End Station Name', 'End Year', 'End Month',
       'End Hour', 'End Day of Week', 'End Holiday', 'End Lat', 'End Lon', 'End Day of Year', 'End Cluster', 
       'End Temp']]

# 2. Function and Class Definitions

In [17]:
def check_weekend(dayofweek):
    if dayofweek > 4:
      return 'weekend'
    else:
      return 'weekday'

In [18]:
merged_data.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type,Bike Id,...,Start Lon,End Lat,End Lon,End Day of Year,Start Cluster,End Cluster,Start Day,End Day,Start Temp,End Temp
0,4581278.0,1547.0,7021.0,2019-01-01 00:08:00,Bay St / Albert St,7233.0,2019-01-01 00:33:00,King / Cowan Ave - SMART,Annual Member,1296.0,...,-79.382458,43.637922,-79.431734,1,0,32,1,1,5.1,5.1
1,4586979.0,1243.0,7021.0,2019-01-03 17:13:00,Bay St / Albert St,7233.0,2019-01-03 17:34:00,King / Cowan Ave - SMART,Annual Member,3168.0,...,-79.382458,43.637922,-79.431734,3,0,32,3,3,1.0,1.0
2,4591641.0,1112.0,7021.0,2019-01-04 21:42:00,Bay St / Albert St,7233.0,2019-01-04 22:00:00,King / Cowan Ave - SMART,Annual Member,3512.0,...,-79.382458,43.637922,-79.431734,4,0,32,4,4,3.9,3.0
3,4594328.0,1156.0,7021.0,2019-01-05 23:21:00,Bay St / Albert St,7233.0,2019-01-05 23:40:00,King / Cowan Ave - SMART,Annual Member,212.0,...,-79.382458,43.637922,-79.431734,5,0,32,5,5,2.6,2.6
4,4596322.0,1068.0,7021.0,2019-01-06 21:42:00,Bay St / Albert St,7233.0,2019-01-06 22:00:00,King / Cowan Ave - SMART,Annual Member,3279.0,...,-79.382458,43.637922,-79.431734,6,0,32,6,6,-4.6,-4.9


In [19]:
len(merged_data['Start Cluster'].unique())

37

In [20]:
class BikeDemand:
  def __init__(self, nTransitions, tempflag):
    self.nTransitionMatrix = nTransitions
    self.tempFlag = tempflag

  # Training: Months 1-9, Testing: Months 10-12
  def train_test_split(self, data, start_end_flag):
    training_data = data.loc[(data['{} Month'.format(start_end_flag)] >= 1) & (data['{} Month'.format(start_end_flag)] <= 9)]
    testing_data = data.loc[(data['{} Month'.format(start_end_flag)] >= 10) & (data['{} Month'.format(start_end_flag)] <= 12)]
    print(data.shape, training_data.shape, testing_data.shape)
    return training_data, testing_data
  
  def fill_in_missing_combinations(self, data, start_end_flag):
    # Check missing combinations
    
    hours = np.arange(24)
    # dayofyear = data['{} Day of Year'.format(start_end_flag)].unique()
    weekday_weekend = ['weekday', 'weekend']
    clusters = np.arange(37)

    combinations = itertools.product(hours, weekday_weekend, clusters)
    features_list = ['{} Hour'.format(start_end_flag), 'weekday/weekend', '{} Cluster'.format(start_end_flag)]
    df_combinations = data[features_list].to_numpy().astype('str')
    df_combinations = df_combinations.tolist()
    comb_list = []
    for comb in combinations:
      comb = [str(comb[0]), comb[1], str(comb[2])]
      if comb not in df_combinations:
        # print(comb)
        comb_list.append(comb)
    print('number of missing combinations:', len(comb_list))

    # Fill in missing combinations
    for comb in comb_list:
      hour = int(comb[0])
      weekday_weekend = comb[1]
      cluster = int(comb[2])
      df_select = data[(data['{} Hour'.format(start_end_flag)] == hour) & (data['weekday/weekend'] == weekday_weekend)]
      
      clusters_list = data['{} Cluster'.format(start_end_flag)].unique()
      for c in clusters:
        if c not in clusters_list:
          if self.tempFlag == True:
            for temperature in df_select['{} Temp'.format(start_end_flag)].unique():
              for doy in df_select[df_select['{} Temp'.format(start_end_flag)] == temperature]['{} Day of Year'.format(start_end_flag)].unique():
                new_row = {'{} Hour'.format(start_end_flag) : hour, 'weekday/weekend' : weekday_weekend, 
                            '{} Cluster'.format(start_end_flag) : cluster, '{} Day of Year'.format(start_end_flag) : doy,
                            '{} Temp'.format(start_end_flag) : temperature, 'Trip Id': 0.000001}
                data = data.append(new_row, ignore_index = True)
          else:
            for doy in df_select['{} Day of Year'.format(start_end_flag)].unique():
                new_row = {'{} Hour'.format(start_end_flag) : hour, 'weekday/weekend' : weekday_weekend, 
                            '{} Cluster'.format(start_end_flag) : cluster, '{} Day of Year'.format(start_end_flag) : doy,
                            'Trip Id': 0.000001}
                data = data.append(new_row, ignore_index = True)
    return data
  
  def preprocess_data(self, train_data, test_data, start_end_flag):
    features_list = ['Trip Id', '{} Hour'.format(start_end_flag), 'weekday/weekend', '{} Cluster'.format(start_end_flag), '{} Day of Year'.format(start_end_flag)]
    grouping_features_list = ['{} Hour'.format(start_end_flag), 'weekday/weekend', '{} Cluster'.format(start_end_flag), '{} Day of Year'.format(start_end_flag)]
    reset_levels_list = [0,1,2,3]
    if self.tempFlag == True:
      features_list += ['{} Temp'.format(start_end_flag)]
      grouping_features_list += ['{} Temp'.format(start_end_flag)]
      reset_levels_list += [4]
    train_data = train_data[features_list]
    train_data = train_data.groupby(by=grouping_features_list).count()
    # print(train_data.head())
    train_data = train_data.reset_index(level=reset_levels_list)
    train_data['{} Hour'.format(start_end_flag)] = train_data['{} Hour'.format(start_end_flag)].astype("int64")

    test_data = test_data[features_list]
    test_data = test_data.groupby(by=grouping_features_list).count()
    test_data = test_data.reset_index(level=reset_levels_list)
    test_data['{} Hour'.format(start_end_flag)] = test_data['{} Hour'.format(start_end_flag)].astype("int64")

    train_data = self.fill_in_missing_combinations(train_data, start_end_flag)
    test_data = self.fill_in_missing_combinations(test_data, start_end_flag)

    if start_end_flag == 'Start':
      name = 'Number of Checkouts'
    else:
      name = 'Number of Checkins'

    train_data = train_data.sort_values(by=grouping_features_list)
    train_data.rename({'Trip Id': name}, axis=1, inplace=True)
    train_data.reset_index(drop=True, inplace=True)

    test_data = test_data.sort_values(by=grouping_features_list)
    test_data.rename({'Trip Id': name}, axis=1, inplace=True)
    test_data.reset_index(drop=True, inplace=True)

    print('Train Data Shape: ', train_data.shape)
    print('Test Data Shape: ', test_data.shape)

    return train_data, test_data

  def predict_checkout(self, checkout_train, checkout_test):
    features_list = ['Start Hour', 'weekday/weekend', 'Start Cluster']
    if self.tempFlag == True:
      features_list += ['Start Temp']
    X_train = checkout_train[features_list]
    X_train = pd.get_dummies(data=X_train, columns=['Start Hour', 'weekday/weekend', 'Start Cluster'], drop_first=True).to_numpy()
    y_train = checkout_train[['Number of Checkouts']].to_numpy()

    X_test = checkout_test[features_list]
    X_test = pd.get_dummies(data=X_test, columns=['Start Hour', 'weekday/weekend', 'Start Cluster'], drop_first=True).to_numpy()
    y_test = checkout_test[['Number of Checkouts']].to_numpy()

    y_train_log, y_test_log = np.log(y_train), np.log(y_test)
    checkout_linreg = LinearRegression().fit(X_train, y_train_log)

    y_train_pred = np.exp(checkout_linreg.predict(X_train))
    y_test_pred = np.exp(checkout_linreg.predict(X_test))

    return X_train, y_train, y_train_pred, X_test, y_test, y_test_pred
  
  def evaluate(self, y_true, y_pred):
    mse = round(mean_squared_error(y_true, y_pred, squared=True),3) # MSE
    rmse = round(mean_squared_error(y_true, y_pred, squared=False),3) # RMSE
    mre = round(max_error(y_true, y_pred),3) # maximum residual error
    mae = round(mean_absolute_error(y_true, y_pred),3) # MAE
    # r2 = round(r2_score(y_true, y_pred),3)
    # return mse, rmse, mre, mae, r2
    return mse, rmse, mre, mae

  def generate_transition_matrix(self, df_train_select):
    from_cluster_list = list(np.arange(0, 37))
    df_transition_matrix = pd.DataFrame(data=np.zeros((37, 37)), columns = from_cluster_list, index=from_cluster_list)
    df_transition_matrix = df_transition_matrix.astype("int")

    for from_cluster in from_cluster_list:
      data_train_temp = df_train_select[df_train_select['Start Cluster'] == from_cluster]
      df_counts = pd.DataFrame(data_train_temp['End Cluster'].value_counts())
      
      # print(df_counts.head())
      to_cluster_list = list(df_counts.index)
      
      for to_cluster in to_cluster_list:
        cnt = df_counts.loc[to_cluster]['End Cluster']
        df_transition_matrix.loc[from_cluster, to_cluster] = cnt
      
      df_transition_matrix = df_transition_matrix.div(df_transition_matrix.sum(axis=1), axis=0)

    df_transition_matrix.replace(np.nan, 0, inplace=True)
    return df_transition_matrix
  
  def generate_transition_matrix_dict(self, data_train):
    n = self.nTransitionMatrix
    transition_matrix_dict = {}
    if n == 1:
      return self.generate_transition_matrix(data_train)
    
    elif n == 24:
      for hr in data_train['Start Hour'].unique():
        df_train_select = data_train[data_train['Start Hour'] == hr]
        transition_matrix = self.generate_transition_matrix(df_train_select)
        transition_matrix_dict[hr] = transition_matrix
      return transition_matrix_dict
    
    elif n == 48:
      for hr in data_train['Start Hour'].unique():
        for wd in data_train['weekday/weekend'].unique():
          df_train_select = data_train[(data_train['Start Hour'] == hr) & (data_train['weekday/weekend'] == wd)]
          transition_matrix = self.generate_transition_matrix(df_train_select)
          transition_matrix_dict[(hr, wd)] = transition_matrix
      return transition_matrix_dict
  
  def predict_checkin_by_cluster(self, df_predictions, df_transition_matrix):
    checkin_prediction_list = []
    columns_list = ['End Cluster {}'.format(c) for c in np.arange(37)]
    df_checkin = pd.DataFrame(columns = columns_list)

    if self.nTransitionMatrix == 1:
      for idx, row in df_predictions.iterrows():
        if idx % 5000 == 0:
          print(idx)
        checkout_prediction = row['Check-out Predictions']
        cluster_n = row['Start Cluster']
        transition_list = np.array(df_transition_matrix.iloc[cluster_n])
        # check-out predictions x transition matrix = check-in predictions
        checkin_prediction = np.round(transition_list * checkout_prediction, 6)
        checkin_prediction = pd.Series(checkin_prediction, index = df_checkin.columns)
        df_checkin = df_checkin.append(checkin_prediction, ignore_index=True)
    elif self.nTransitionMatrix == 24:
      for idx, row in df_predictions.iterrows():
        if idx % 5000 == 0:
          print(idx)
        checkout_prediction = row['Check-out Predictions']
        cluster_n = row['Start Cluster']
        hour = row['Start Hour']
        transition_list = np.array(df_transition_matrix[hour].iloc[cluster_n])
        # check-out predictions x transition matrix = check-in predictions
        checkin_prediction = np.round(transition_list * checkout_prediction, 6)
        checkin_prediction = pd.Series(checkin_prediction, index = df_checkin.columns)
        df_checkin = df_checkin.append(checkin_prediction, ignore_index=True)
    elif self.nTransitionMatrix == 48:
      for idx, row in df_predictions.iterrows():
        if idx % 5000 == 0:
          print(idx)
        checkout_prediction = row['Check-out Predictions']
        cluster_n = row['Start Cluster']
        hour, weekday = row['Start Hour'], row['weekday/weekend']
        transition_list = np.array(df_transition_matrix[(hour, weekday)].iloc[cluster_n])
        # check-out predictions x transition matrix = check-in predictions
        checkin_prediction = np.round(transition_list * checkout_prediction, 6)
        checkin_prediction = pd.Series(checkin_prediction, index = df_checkin.columns)
        df_checkin = df_checkin.append(checkin_prediction, ignore_index=True)
    
    return pd.concat([df_predictions, df_checkin], axis=1)
  
  def compute_true_checkin(self, checkin_test):
    features_list = ['End Hour', 'weekday/weekend', 'End Day of Year']
    data_columns = ['End Hour','weekday/weekend', 'End Cluster', 'Number of Checkins']
    if self.tempFlag == True:
      features_list += ['End Temp']
      data_columns += ['End Temp']
    df = checkin_test[features_list].drop_duplicates()
    clusters = np.arange(37)
    df_checkin = pd.DataFrame(columns = data_columns)
    for idx, row in df.iterrows():
      if self.tempFlag == True:
        hr, wd, edoy, temp = row[0], row[1], row[2], row[3]
        df_temp = checkin_test[(checkin_test['End Hour'] == hr) & (checkin_test['weekday/weekend'] == wd) & 
                             (checkin_test['End Day of Year'] == edoy) & (checkin_test['End Temp'] == temp)]
      else: 
        hr, wd, edoy = row[0], row[1], row[2]
        df_temp = checkin_test[(checkin_test['End Hour'] == hr) & (checkin_test['weekday/weekend'] == wd) & 
                             (checkin_test['End Day of Year'] == edoy)]
      
      clusters_list = df_temp['End Cluster'].unique()
      for c in clusters:
        if c not in clusters_list:
          print(c)
          if self.tempFlag == True:
            new_row = {'End Hour' : hr, 'weekday/weekend' : wd, 'End Cluster' : c, 'End Day of Year' : edoy, 'End Temp' : temp, 'Number of Checkins': 0.000001}
          else:
            new_row = {'End Hour' : hr, 'weekday/weekend' : wd, 'End Cluster' : c, 'End Day of Year' : edoy, 'Number of Checkins': 0.000001}
          df_temp = df_temp.append(new_row, ignore_index = True)
      if self.tempFlag == True:
        df_temp.sort_values(by=['End Hour', 'weekday/weekend', 'End Cluster', 'End Day of Year', 'End Temp'], inplace=True)
      else:
        df_temp.sort_values(by=['End Hour', 'weekday/weekend', 'End Cluster', 'End Day of Year'], inplace=True)
      df_temp.reset_index(drop=True, inplace=True)
      if len(df_temp) != 37:
        print(df_temp)
      df_checkin = df_checkin.append(df_temp, ignore_index = True)
        
    return df_checkin
  
  def run_prediction_pipeline(self, merged_data, checkout_data, checkin_data):
    print('Checkout predictions')
    checkout_train, checkout_test = self.train_test_split(checkout_data, 'Start')
    checkout_train, checkout_test = self.preprocess_data(checkout_train, checkout_test, 'Start')
    X_train, y_train, y_train_pred, X_test, y_test, y_test_pred = self.predict_checkout(checkout_train, checkout_test)

    print('Checkout training and testing errors')
    mse_train, rmse_train, mre_train, mae_train = self.evaluate(y_train, y_train_pred)
    mse_test, rmse_test, mre_test, mae_test = self.evaluate(y_test, y_test_pred)
    print('training errors: ', mse_train, rmse_train, mre_train, mae_train)
    print('testing errors: ', mse_test, rmse_test, mre_test, mae_test)

    print('Transition Matrix Computation')
    data_train, data_test = self.train_test_split(merged_data, 'Start')
    data_train['weekday/weekend'] = data_train.apply(lambda row: check_weekend(row['Start Day of Week']), axis=1)
    df_transition_matrix = self.generate_transition_matrix_dict(data_train)

    print('Summarize checkout predictions')
    if self.tempFlag == True:
      df_predictions = checkout_test[['Start Hour', 'weekday/weekend', 'Start Cluster', 'Start Temp']]
    else:
      df_predictions = checkout_test[['Start Hour', 'weekday/weekend', 'Start Cluster']]
    df_predictions['Check-out Predictions'], df_predictions['Check-out True Values'] = y_test_pred, y_test
    df_predictions = self.predict_checkin_by_cluster(df_predictions, df_transition_matrix)

    print('Checkin predictions')
    checkin_train, checkin_test = self.train_test_split(checkin_data, 'End')
    checkin_train, checkin_test = self.preprocess_data(checkin_train, checkin_test, 'End')
    df_checkin = self.compute_true_checkin(checkin_test)
    checkin_pred = []
    
    if self.tempFlag == True:
      df_checkin_row = df_checkin[['End Hour', 'weekday/weekend', 'End Temp', 'End Day of Year']].drop_duplicates()
    else:
      df_checkin_row = df_checkin[['End Hour', 'weekday/weekend', 'End Day of Year']].drop_duplicates()
    
    for idx, row in df_checkin_row.iterrows():
      if self.tempFlag == True:
        hr, wd, temp = row[0], row[1], row[2]
        temp_pred_list = df_predictions[(df_predictions['Start Hour'] == hr) & (df_predictions['weekday/weekend'] == wd) & (df_predictions['Start Temp'] == temp)].iloc[:, 6:].sum(axis=0)
      else:
        hr, wd = row[0], row[1]
        temp_pred_list = df_predictions[(df_predictions['Start Hour'] == hr) & (df_predictions['weekday/weekend'] == wd)].iloc[:, 5:].sum(axis=0)
      
      if len(temp_pred_list) != 37:
        print(len(temp_pred_list))
      checkin_pred += temp_pred_list.to_list()
    df_checkin['Predicted Number of Checkins'] = checkin_pred

    print('Evaluate the predictions')
    checkout_true = df_predictions['Check-out True Values'].to_numpy()
    checkout_pred = df_predictions['Check-out Predictions'].to_numpy()
    checkin_true = df_checkin[['Number of Checkins']].to_numpy()
    checkin_pred = df_checkin[['Predicted Number of Checkins']].to_numpy()

    print('Checkout evaluation:')
    checkout_mse, checkout_rmse, checkout_mre, checkout_mae = self.evaluate(checkout_true, checkout_pred)
    print('mse, rmse, mre, mae: ', checkout_mse, checkout_rmse, checkout_mre, checkout_mae)
    print('Checkin evaluation:')
    checkin_mse, checkin_rmse, checkin_mre, checkin_mae = self.evaluate(checkin_true, checkin_pred)
    print('mse, rmse, mre, mae: ', checkin_mse, checkin_rmse, checkin_mre, checkin_mae)
    return df_transition_matrix, checkout_mse, checkout_rmse, checkout_mre, checkout_mae, checkin_mse, checkin_rmse, checkin_mre, checkin_mae

# 3. Run Pipeline

In [21]:
# Checkout data
checkout_data['weekday/weekend'] = checkout_data.apply(lambda row: check_weekend(row['Start Day of Week']), axis=1)
# Checkin data
checkin_data['weekday/weekend'] = checkin_data.apply(lambda row: check_weekend(row['End Day of Week']), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [23]:
start_time = time.time()
bikedemand = BikeDemand(nTransitions=1, tempflag=False)
df_transition_matrix, checkout_mse, checkout_rmse, checkout_mre, checkout_mae, checkin_mse, checkin_rmse, checkin_mre, checkin_mae = bikedemand.run_prediction_pipeline(merged_data, checkout_data, checkin_data)
print("Total Time Elapsed: ", time.time() - start_time)

Checkout predictions
(2438720, 16) (1970776, 16) (467944, 16)
number of missing combinations: 21
number of missing combinations: 74
Train Data Shape:  (154045, 5)
Test Data Shape:  (51015, 5)
Checkout training and testing errors
training errors:  266.071 16.312 332.359 8.287
testing errors:  111.762 10.572 187.359 5.493
Transition Matrix Computation
(2438720, 33) (1970776, 33) (467944, 33)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Summarize checkout predictions
0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
Checkin predictions
(2438720, 15) (1970751, 15) (467969, 15)
number of missing combinations: 16
number of missing combinations: 63
Train Data Shape:  (152449, 5)
Test Data Shape:  (50390, 5)


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Evaluate the predictions
Checkout evaluation:
mse, rmse, mre, mae:  111.762 10.572 187.359 5.493
Checkin evaluation:
mse, rmse, mre, mae:  229102.479 478.647 2622.218 278.075
Total Time Elapsed:  395.0407636165619


In [ ]:
df_transition_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,0.088817,0.023390,0.025316,0.073562,0.007252,0.044845,0.109464,0.005082,0.098633,0.029017,0.000855,0.000451,0.009816,0.000601,0.031469,0.003307,0.062148,0.000122,0.000000,0.002809,0.000958,0.004387,0.000413,0.000892,0.002151,0.005768,0.016345,0.000094,0.000751,0.009563,0.084900,0.027382,0.013048,0.060345,0.083660,0.067287,0.005101
1,0.089650,0.082856,0.020639,0.079395,0.042090,0.045979,0.075763,0.006709,0.063499,0.018802,0.000427,0.001667,0.005128,0.002436,0.013332,0.004615,0.027733,0.000000,0.000000,0.000427,0.000214,0.000470,0.000171,0.000983,0.021537,0.047945,0.096402,0.000897,0.012392,0.002692,0.063413,0.013204,0.001966,0.015469,0.102213,0.037219,0.001667
2,0.069891,0.013799,0.086898,0.041862,0.012747,0.071857,0.131503,0.002949,0.082224,0.084414,0.002794,0.002156,0.014472,0.006175,0.062785,0.010384,0.067304,0.001414,0.000052,0.003536,0.000000,0.000604,0.000793,0.000949,0.000690,0.022216,0.006692,0.000673,0.005002,0.001794,0.051470,0.010470,0.006175,0.015524,0.063268,0.042621,0.001846
3,0.129491,0.035993,0.025833,0.092258,0.017434,0.049105,0.077483,0.016031,0.103201,0.015575,0.000196,0.000489,0.005121,0.000261,0.019097,0.001174,0.047165,0.000114,0.000082,0.000620,0.000424,0.000962,0.000196,0.000522,0.008627,0.012982,0.042452,0.001859,0.002593,0.007078,0.069801,0.015819,0.008073,0.031981,0.120929,0.035276,0.003702
4,0.060706,0.069123,0.040162,0.073033,0.090662,0.078799,0.111339,0.001789,0.063159,0.034727,0.000596,0.000464,0.004109,0.001657,0.033998,0.001591,0.058321,0.000066,0.000663,0.002054,0.000133,0.000133,0.000066,0.000199,0.004440,0.103652,0.027835,0.001789,0.013188,0.000663,0.053947,0.003380,0.000928,0.006230,0.037179,0.017894,0.001325
5,0.079505,0.019725,0.054262,0.053731,0.020013,0.098259,0.156706,0.002941,0.065981,0.047091,0.000500,0.000197,0.009006,0.001925,0.046181,0.002638,0.099882,0.000136,0.000030,0.001744,0.000030,0.000758,0.000212,0.000591,0.001501,0.019634,0.011386,0.000652,0.003654,0.001455,0.053458,0.007672,0.005352,0.008475,0.088966,0.033855,0.001895
6,0.109071,0.014444,0.045332,0.039070,0.011428,0.077166,0.146378,0.002522,0.047718,0.053686,0.001103,0.000795,0.017833,0.001598,0.072394,0.004779,0.083929,0.000279,0.000000,0.004263,0.000294,0.002092,0.000215,0.001354,0.000688,0.011306,0.005761,0.000129,0.001333,0.002680,0.066268,0.013678,0.005288,0.020807,0.082503,0.049223,0.002594
7,0.047853,0.010344,0.012329,0.117020,0.004075,0.015568,0.029986,0.139693,0.128931,0.003761,0.000104,0.000000,0.002194,0.000313,0.006164,0.000627,0.007418,0.000104,0.000000,0.000627,0.000522,0.000627,0.000104,0.000522,0.022673,0.004806,0.176784,0.000104,0.001254,0.003970,0.024240,0.007836,0.002612,0.066451,0.139484,0.020061,0.000836
8,0.114424,0.021311,0.027679,0.088210,0.010597,0.041666,0.064076,0.019468,0.105021,0.013018,0.000236,0.000083,0.005505,0.000461,0.015854,0.001949,0.032723,0.000024,0.000024,0.000650,0.000839,0.002918,0.000201,0.000543,0.005576,0.007891,0.039031,0.000130,0.000721,0.014034,0.069392,0.020862,0.008139,0.070715,0.152298,0.039917,0.003816
9,0.045938,0.005674,0.055292,0.014675,0.006779,0.034168,0.111540,0.000444,0.023208,0.129599,0.008146,0.005059,0.031035,0.007782,0.140274,0.034225,0.113067,0.003065,0.000057,0.010402,0.000194,0.001048,0.001903,0.004295,0.000513,0.014219,0.002632,0.000741,0.002267,0.002563,0.049903,0.012065,0.005560,0.016281,0.056214,0.047077,0.002096
